In [1]:
%load_ext autoreload
%autoreload 2

import loading
import pandas as pd
import numpy as np
import ipaddress

## Load selected logs for processing
Some logs are not used, because they describe the zeek session and not the data.

We decided to not use x509.log, since the data pertains to a very small portion of flows and most columns dont seem that important (type of eliptic curve? etc.). This log captures details on certificates exchanged during certain TLS negotiations. The dataset is very small and I wanted to prioritize hand-picked high-impact features, so the ML models used later converge well and dont struggle with excess dimensionality. If I had much larger dataset, I would more carefully consider every data for importance or try using different feature selection techniques to reduce the number columns.
https://docs.zeek.org/en/master/logs/x509.html

In [2]:
ignored_logs = [ # some of the logs contain information about the zeek session and are not useful for the analysis
    "loaded_scripts.log",
    "capture_loss.log",
    "stats.log",
    "packet_filter.log",
    "x509.log",
]
zeek_logs = loading.load_all_zeek_logs("../stratosphere-work-challenge-v1/zeek",ignored_logs) # change this to the path of the zeek logs

print(zeek_logs.keys())
cleaned_logs = {} #where cleaned and aggregated data will be stored


Loading conn.log...
Loading dns.log...
Loading files.log...
Loading http.log...
Loading ssl.log...

dict_keys(['conn', 'dns', 'files', 'http', 'ssl'])


## Extracting info from individual logs
We select only a small number of important columns (based on prior knowledge AND inspecting the logs themselves). Some columns would be selected, but they have only one unique value in the respective log.

In [3]:
# files log
# multiple fuids for one uid. cannot connect 1:1, I want some kind of aggregation
files_df = zeek_logs["files"]
http_count = []
ssl_count = []
avg_seen_bytes = []

# for each uid, count the number of http and ssl files and calculate the average seen_bytes
for uid in files_df['conn_uids'].unique():
    http_files = files_df[(files_df['conn_uids'] == uid) & (files_df['source'] == 'HTTP')]
    ssl_files = files_df[(files_df['conn_uids'] == uid) & (files_df['source'] == 'SSL')]
    
    http_count.append(len(http_files))
    ssl_count.append(len(ssl_files))
    avg_seen_bytes.append(files_df[files_df['conn_uids'] == uid]['seen_bytes'].mean())

aggregated_file_df = pd.DataFrame({
    'uid': files_df['conn_uids'].unique(),
    'http_count': http_count,
    'ssl_count': ssl_count,
    'avg_seen_bytes_files': avg_seen_bytes
})

print(aggregated_file_df.head())
cleaned_logs["files"] = aggregated_file_df # no missing values and no duplicates

                  uid  http_count  ssl_count  avg_seen_bytes_files
0  CYaRbd1LgVHyMi0os7           0          2                1392.0
1  CE2v1V1PiJwfenwq22           0          2                1392.0
2   CnCVN6i60NAbKmFxl           0          2                1392.0
3   CXVBUUhqkJEhiN2s6           0          2                1424.0
4  CriN9h1d6hCNTMi3P6           1          0                  22.0


In [4]:
# ssl contains info about https (but in conn, there already is tcp with ssl tags)
ssl = zeek_logs["ssl"]
aggregated_ssl_df = ssl[
    ["uid", "resumed", "established", "validation_status"]
]

aggregated_ssl_df['validation_status'] = aggregated_ssl_df['validation_status'].cat.add_categories("not_provided")
aggregated_ssl_df['validation_status'] = aggregated_ssl_df['validation_status'].fillna("not_provided")

cleaned_logs["ssl"] = aggregated_ssl_df
print(aggregated_ssl_df)

                   uid resumed established validation_status
0    CgnCcKyQn6Fuvtaaa       T           T      not_provided
1    CyaZhP168fSnjEW4i       T           T      not_provided
2   C1joeV1VXsHCLjiswg       T           T      not_provided
3    CplZRBVCFLcGq4Dbl       T           T      not_provided
4    C05gpYcCZjI5tZ4m5       T           T      not_provided
..                 ...     ...         ...               ...
73  CoqeK14slRRWT3jSG3       F           T      not_provided
74   Cmz3aX3s2GpoaPURW       T           T      not_provided
75  Ci1SH62mLUPecPJx5d       T           T      not_provided
76   CupYkjJ1BXaXaZk94       T           T      not_provided
77  CokYEW2q0cjgDIE5Gi       T           T      not_provided

[78 rows x 4 columns]


/tmp/ipykernel_265054/2156740250.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_ssl_df['validation_status'] = aggregated_ssl_df['validation_status'].cat.add_categories("not_provided")
/tmp/ipykernel_265054/2156740250.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_ssl_df['validation_status'] = aggregated_ssl_df['validation_status'].fillna("not_provided")


In [5]:
# http log 
http = zeek_logs["http"]
aggregated_http_df = http[
    ["uid","method", "status_code", "request_body_len", "response_body_len"]]
aggregated_http_df['status_code'] = aggregated_http_df['status_code'].fillna(0).astype('category')

cleaned_logs["http"] = aggregated_http_df
print(aggregated_http_df)

                   uid method status_code  request_body_len  response_body_len
0   CBPoVA3CEo9RnTQNDj    GET         204                 0                  0
1   CriN9h1d6hCNTMi3P6    GET         200                 0                 22
2   CKwfPy2GoRYlUeD35i    GET         200                 0                101
3   CKwfPy2GoRYlUeD35i    GET         200                 0                  5
4   CKwfPy2GoRYlUeD35i    GET         200                 0                  4
5   Cpl4DO1V4VtiWE0oBj    GET         200                 0                101
6   Cpl4DO1V4VtiWE0oBj    GET         200                 0                  5
7   Cpl4DO1V4VtiWE0oBj    GET         200                 0                  4
8    C0Uxcf30mJ2Lq5wbX    GET         101                 0                  0
9   CNPVre20YbYQSMf7Ke    GET         101                 0                  0
10  Ctq0ZM2pGM8l5HdQS6    GET         200                 0                101
11  CtCYjh1iQvaBV7zgZ1    GET         200           

/tmp/ipykernel_265054/4248506904.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_http_df['status_code'] = aggregated_http_df['status_code'].fillna(0).astype('category')


In [6]:
#dns log
dns = zeek_logs["dns"]
dns_df = dns[["uid", "qtype_name", "rcode_name"]]
dns_answer_count = dns['answers'].apply(lambda x: len(x.split(',')) if pd.notna(x) else 0).astype(int)
dns_df['dns_answer_count'] = dns_answer_count # number of dns answers

# calculate average ttl of answer packet (can mean very fast name change? fast-flux servers)
dns_df['avg_TTL'] = dns['TTLs'].apply(lambda x: np.mean([float(ttl) for ttl in x.split(',')]) if pd.notna(x) else 0)


# extract TLD (instead of the full query)
dns_df["TLD"] = dns["query"].apply(lambda x: x.split(".")[-1])
dns_df["TLD"] = dns_df["TLD"].astype("category")

# extract SLD (google in google.com ...)
dns_df["SLD"] = dns["query"].apply(lambda x: x.split(".")[-2] if len(x.split(".")) > 1 else "")
# Frequency encoding for SLD (many values for sld, I want at least their relative frequency)
sld_counts = dns_df['SLD'].value_counts()
dns_df['SLD_freq'] = dns_df['SLD'].map(sld_counts)
dns_df.drop(columns=['SLD'],inplace=True)

# number of subdomains in the query (longer can be used for obfuscation)
dns_df["query_length"] = dns["query"].apply(lambda x: len(x.split(".")))

print(dns_df)
cleaned_logs["dns"] = dns_df

                   uid qtype_name rcode_name  dns_answer_count  avg_TTL  TLD  \
0   CMLZbu3FDJYoZwa27k          A    NOERROR                 2    116.5  com   
1   CgRCjV3z8dKmNVIvhb          A    NOERROR                 2   1800.5  com   
2   CgFfWv3PUApAZUINNf          A    NOERROR                 1     59.0  com   
3   CIkFu02IznJPZcp1El          A    NOERROR                 2   1814.5  com   
4   Cylq6E2mc9lVLjs8ua          A    NOERROR                 2    636.5  com   
..                 ...        ...        ...               ...      ...  ...   
59  Cyw1E94W8FfeSPczJk          A    NOERROR                 1    299.0  com   
60  C0jMGu1frh9ei1Wmqc          A    NOERROR                 1    247.0  com   
61  COs0MF2BvtxqwwWZBl          A    NOERROR                 1    248.0  com   
62  CovR8F24TUZtn0RYGj          A    NOERROR                 1    576.0   co   
63   Cj4sBHa0s2uYpSCc3          A    NOERROR                 5   1793.0  com   

    SLD_freq  query_length  
0         

/tmp/ipykernel_265054/2788785345.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dns_df['dns_answer_count'] = dns_answer_count # number of dns answers
/tmp/ipykernel_265054/2788785345.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dns_df['avg_TTL'] = dns['TTLs'].apply(lambda x: np.mean([float(ttl) for ttl in x.split(',')]) if pd.notna(x) else 0)
/tmp/ipykernel_265054/2788785345.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [7]:
conn = zeek_logs["conn"]
# here we only drop some of the columns, most are useful and we need stuff like IPs..)
conn.drop(columns=["local_orig", "local_resp","history", "tunnel_parents","log_type"], inplace=True)

# Create a new binary column 'is_icmp' based on the 'proto' column. udp and tcp have other information in "service"
conn["is_icmp"] = np.where(conn["proto"] == "icmp", 1, 0)

conn["id.orig_h"] = conn["id.orig_h"].apply(lambda x: int(ipaddress.ip_address(x)))
conn["id.resp_h"] = conn["id.resp_h"].apply(lambda x: int(ipaddress.ip_address(x)))

# Convert IP address columns to numerical
conn["id.orig_h"] = conn["id.orig_h"].astype(np.int32)
conn["id.resp_h"] = conn["id.resp_h"].astype(np.int32)

conn.drop(columns=["proto"], inplace=True)
print(conn.columns)
cleaned_logs["conn"] = conn

Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state',
       'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts',
       'resp_ip_bytes', 'is_icmp'],
      dtype='object')


In [8]:
# still, throw away columns with only one unique value from any log
for log_name, log_df in cleaned_logs.items():
    for col in log_df.columns:
        if log_df[col].nunique() == 1:
            print(f"{log_name} - {col}: {log_df[col].nunique()} unique value")
            log_df.drop(columns=[col], inplace=True)

# finally merge logs
merged_df = loading.merge_logs(
    cleaned_logs, primary_log="conn"
)  
merged_df.drop(columns=["uid"], inplace=True)

# results?
print("flows in merged log file: ", len(merged_df))
print("columns: ", len(merged_df.columns), merged_df.columns)


dns - qtype_name: 1 unique value
dns - rcode_name: 1 unique value
conn - missed_bytes: 1 unique value
Merging files...
Merging ssl...
Merging http...
Merging dns...
flows in merged log file:  234
columns:  30 Index(['ts', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'service',
       'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'orig_pkts',
       'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'is_icmp', 'http_count',
       'ssl_count', 'avg_seen_bytes_files', 'resumed', 'established',
       'validation_status', 'method', 'status_code', 'request_body_len',
       'response_body_len', 'dns_answer_count', 'avg_TTL', 'TLD', 'SLD_freq',
       'query_length'],
      dtype='object')


/tmp/ipykernel_265054/1816079443.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log_df.drop(columns=[col], inplace=True)
/tmp/ipykernel_265054/1816079443.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log_df.drop(columns=[col], inplace=True)


## Further clean-up the data 
#### Label the data and drop the time columns

In [9]:
print("calculate time of the start w.r.t. the first log entry and label the data as benign if the time is less than 4 minutes")# based on task description
merged_df['time_from_beginning'] = (merged_df['ts'] - merged_df['ts'].min()).dt.total_seconds()
merged_df["label"] = np.where(
    merged_df["time_from_beginning"] < 4 * 60, "benign", "unknown"
)
merged_df['label'] = merged_df['label'].astype('category')
merged_df.drop(columns=["ts", "time_from_beginning"], inplace=True)

calculate time of the start w.r.t. the first log entry and label the data as benign if the time is less than 4 minutes


Lets see the columns, their names and how many unique values do they have. we should have numerical columns, some categories and duration which is a time column.

In [10]:
print("column types: ")
for col in merged_df.columns:
    print(col, merged_df[col].dtype, merged_df[col].nunique())

column types: 
id.orig_h int32 3
id.orig_p UInt16 170
id.resp_h int32 41
id.resp_p UInt16 8
service category 3
duration timedelta64[ns] 189
orig_bytes UInt64 116
resp_bytes UInt64 131
conn_state category 8
orig_pkts UInt64 50
orig_ip_bytes UInt64 125
resp_pkts UInt64 46
resp_ip_bytes UInt64 133
is_icmp int64 2
http_count float64 5
ssl_count float64 3
avg_seen_bytes_files float64 18
resumed category 2
established category 2
validation_status category 2
method category 2
status_code category 4
request_body_len UInt64 2
response_body_len UInt64 6
dns_answer_count float64 6
avg_TTL float64 57
TLD category 4
SLD_freq float64 7
query_length float64 4
label category 2


In [11]:
print("Number of columns in cleaned_df:", len(merged_df.columns))
merged_df.describe(include="all")

Number of columns in cleaned_df: 30


,id.orig_h,id.orig_p,id.resp_h,id.resp_p,service,duration,orig_bytes,resp_bytes,conn_state,orig_pkts,...,method,status_code,request_body_len,response_body_len,dns_answer_count,avg_TTL,TLD,SLD_freq,query_length,label
count,2.340000e+02,234.0,2.340000e+02,234.0,167,201,201.0,201.0,234,234.0,...,25,25,25.0,25.0,64.000000,64.000000,64,64.000000,64.000000,234
unique,NaN,<NA>,NaN,<NA>,3,NaN,<NA>,<NA>,8,<NA>,...,2,4,<NA>,<NA>,NaN,NaN,4,NaN,NaN,2
top,NaN,<NA>,NaN,<NA>,ssl,NaN,<NA>,<NA>,SF,<NA>,...,GET,200,<NA>,<NA>,NaN,NaN,com,NaN,NaN,unknown
freq,NaN,<NA>,NaN,<NA>,78,NaN,<NA>,<NA>,149,<NA>,...,23,18,<NA>,<NA>,NaN,NaN,54,NaN,NaN,132
mean,1.867825e+08,42676.0,-5.680145e+08,3333.273504,NaN,0 days 00:01:16.139389960,6151.945274,176449.074627,NaN,71.837607,...,NaN,NaN,0.32,34.72,2.203125,2813.152604,NaN,7.593750,3.250000,NaN
std,1.404771e+08,16161.352785,1.125869e+09,7954.089616,NaN,0 days 00:02:53.423000225,49674.233238,1362814.758861,NaN,491.278309,...,NaN,NaN,1.10755,46.602504,1.335471,5715.046623,NaN,4.904545,0.503953,NaN
min,1.682964e+08,3.0,-1.826598e+09,4.0,NaN,0 days 00:00:00.000043,0.0,0.0,NaN,0.0,...,NaN,NaN,0.0,0.0,1.000000,17.000000,NaN,1.000000,2.000000,NaN
25%,1.682966e+08,38682.0,-1.826598e+09,53.0,NaN,0 days 00:00:00.011053,38.0,66.0,NaN,1.0,...,NaN,NaN,0.0,0.0,1.000000,247.500000,NaN,4.000000,3.000000,NaN
50%,1.682966e+08,47811.0,1.347441e+08,443.0,NaN,0 days 00:00:03.260685,517.0,494.0,NaN,3.0,...,NaN,NaN,0.0,5.0,2.000000,927.583333,NaN,6.500000,3.000000,NaN
75%,1.682966e+08,52183.5,1.347441e+08,8000.0,NaN,0 days 00:01:05.023765,1485.0,3418.0,NaN,15.0,...,NaN,NaN,0.0,101.0,3.000000,1815.375000,NaN,10.000000,3.250000,NaN


#### Fill missing

In [12]:
#which columns do have empty data?
print(merged_df.columns[merged_df.isna().any()].tolist())

['service', 'duration', 'orig_bytes', 'resp_bytes', 'http_count', 'ssl_count', 'avg_seen_bytes_files', 'resumed', 'established', 'validation_status', 'method', 'status_code', 'request_body_len', 'response_body_len', 'dns_answer_count', 'avg_TTL', 'TLD', 'SLD_freq', 'query_length']


In [13]:
merged_df["orig_bytes"].fillna(0.0, inplace=True)
merged_df["resp_bytes"].fillna(0.0, inplace=True)
merged_df["duration"].fillna(0.0, inplace=True)
merged_df["http_count"].fillna(0.0, inplace=True)
merged_df["ssl_count"].fillna(0.0, inplace=True)
merged_df["avg_seen_bytes_files"].fillna(0.0, inplace=True)
merged_df["request_body_len"].fillna(0.0, inplace=True)
merged_df["response_body_len"].fillna(0.0, inplace=True)
merged_df["dns_answer_count"].fillna(0.0, inplace=True)
merged_df["avg_TTL"].fillna(0.0, inplace=True)
merged_df["SLD_freq"].fillna(0.0, inplace=True)
merged_df["query_length"].fillna(0.0, inplace=True)

/tmp/ipykernel_265054/869725158.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df["orig_bytes"].fillna(0.0, inplace=True)
/tmp/ipykernel_265054/869725158.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

In [14]:
print(merged_df.columns[merged_df.isna().any()].tolist())
for col in merged_df.columns[merged_df.isna().any()]:
    print(f"Unique values in column {col}: {merged_df[col].unique()}")


['service', 'resumed', 'established', 'validation_status', 'method', 'status_code', 'TLD']
Unique values in column service: ['dns', 'ssl', NaN, 'http']
Categories (3, object): ['dns', 'http', 'ssl']
Unique values in column resumed: [NaN, 'T', 'F']
Categories (2, object): ['F', 'T']
Unique values in column established: [NaN, 'T', 'F']
Categories (2, object): ['F', 'T']
Unique values in column validation_status: [NaN, 'not_provided', 'ok']
Categories (2, object): ['ok', 'not_provided']
Unique values in column method: [NaN, 'GET', 'POST']
Categories (2, object): ['GET', 'POST']
Unique values in column status_code: [NaN, 204, 200, 101, 0]
Categories (4, UInt64): [0, 101, 200, 204]
Unique values in column TLD: ['com', 'net', NaN, 'co', 'ms']
Categories (4, object): ['co', 'com', 'ms', 'net']


In [15]:
# for all the categories, those are categories from respective logs, which just dont exist for all flows (like type of http request)
# will fill missing valies for those, but are largely redundant, since if you dont have one, you dont have other from the same log. 

# service column
merged_df['service'] = merged_df['service'].cat.add_categories("no_service")
merged_df['service'].fillna("no_service", inplace=True)

# resumed column
merged_df['resumed'] = merged_df['resumed'].cat.add_categories("missing")
merged_df['resumed'].fillna("missing", inplace=True)

# established column
merged_df['established'] = merged_df['established'].cat.add_categories("missing")
merged_df['established'].fillna("missing", inplace=True)

# validation_status column
merged_df['validation_status'] = merged_df['validation_status'].cat.add_categories("missing")
merged_df['validation_status'].fillna("missing", inplace=True)

#method column
merged_df['method'] = merged_df['method'].cat.add_categories("no_method")
merged_df['method'].fillna("no_method", inplace=True)

# status code column
merged_df['status_code'] = merged_df['status_code'].cat.add_categories("missing")
merged_df['status_code'].fillna("missing", inplace=True)

#tld
merged_df['TLD'] = merged_df['TLD'].cat.add_categories("missing")
merged_df['TLD'].fillna("missing", inplace=True)

/tmp/ipykernel_265054/4286792628.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['service'].fillna("no_service", inplace=True)
/tmp/ipykernel_265054/4286792628.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

#### Dummy columns for categories

In [16]:
categorical_columns = merged_df.select_dtypes(include=['category']).columns

for col in categorical_columns:
    print(f"{col}: {merged_df[col].nunique()} unique values")

service: 4 unique values
conn_state: 8 unique values
resumed: 3 unique values
established: 3 unique values
validation_status: 3 unique values
method: 3 unique values
status_code: 5 unique values
TLD: 5 unique values
label: 2 unique values


In [19]:
# Generate dummy columns for all categorical columns
merged_df = pd.get_dummies(merged_df, columns=categorical_columns, drop_first=True)

print("Number of columns after generating dummy variables:", len(merged_df.columns))
print("Columns:", merged_df.columns)

Number of columns after generating dummy variables: 48
Columns: Index(['id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'duration',
       'orig_bytes', 'resp_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts',
       'resp_ip_bytes', 'is_icmp', 'http_count', 'ssl_count',
       'avg_seen_bytes_files', 'request_body_len', 'response_body_len',
       'dns_answer_count', 'avg_TTL', 'SLD_freq', 'query_length',
       'service_http', 'service_ssl', 'service_no_service', 'conn_state_RSTO',
       'conn_state_RSTR', 'conn_state_S0', 'conn_state_S1', 'conn_state_S3',
       'conn_state_SF', 'conn_state_SH', 'resumed_T', 'resumed_missing',
       'established_T', 'established_missing',
       'validation_status_not_provided', 'validation_status_missing',
       'method_POST', 'method_no_method', 'status_code_101', 'status_code_200',
       'status_code_204', 'status_code_missing', 'TLD_com', 'TLD_ms',
       'TLD_net', 'TLD_missing', 'label_unknown'],
      dtype='object')


## Save the dataset to csv

In [20]:
print(f"Number of columns: {merged_df.shape[1]}")
print(f"Number of flows: {merged_df.shape[0]}")

Number of columns: 48
Number of flows: 234


In [21]:
# save for inspection?
merged_df.to_csv("merged_zeek_logs.csv", index=False)